In [1]:
import os
import pandas as pd
import json
from PIL import Image, ImageEnhance
from fpdf import FPDF

In [2]:
CSV_FILE = 'samsung_products.csv'
IMAGE_DIR = 'images'
OUTPUT_DIR = 'output'

ENHANCED_IMG_DIR = os.path.join(OUTPUT_DIR, 'enhanced_images')
JSON_OUTPUT = os.path.join(OUTPUT_DIR, 'enriched_data.json')
PDF_OUTPUT = os.path.join(OUTPUT_DIR, 'product_brochure.pdf')

os.makedirs(ENHANCED_IMG_DIR, exist_ok=True)


In [3]:
df = pd.read_csv(CSV_FILE)
df['ID'] = range(1, len(df) + 1)

In [4]:
df.head(2)

,Product,Weight,Dimensions,Battery Life,Color,Display,Processor,Camera,Water Resistance,Material,Price,Connectivity,Storage,Features,ID
0,Galaxy S23,168g,146.3 x 70.9 x 7.6 mm,20 hours,Phantom Black,"6.1 inch Dynamic AMOLED, 2340x1080, 120Hz",Snapdragon 8 Gen 2,"50MP main, 10MP telephoto, 12MP ultra-wide, 12...",IP68,Armor Aluminum,$799,5G,128GB,"Wireless charging, IP68 certified, 5G",1
1,QLED TV Q90,18 kg,1230 x 710 x 60 mm,NaN,Matte Black,"65 inch 4K UHD QLED, HDR10+",Samsung Quantum Processor,NaN,NaN,Metal & Plastic,$1500,Wi-Fi 5,NaN,"Smart TV, Voice Control, HDR10+",2


In [8]:
def normalize_weight(weight):
    if isinstance(weight, str):
        weight = weight.lower().strip()
        if 'kg' in weight:
            return round(float(weight.replace('kg', '').strip()), 3)
        elif 'g' in weight:
            return round(float(weight.replace('g', '').strip()) / 1000, 3)
    return None


In [9]:
def normalize_price(price):
    try:
        usd_price = float(str(price).replace('$', '').strip())
        inr_price = int(usd_price * 83)  # USD to INR
        return f"Rs.{inr_price}"
    except:
        return None


In [10]:
def normalize_dimensions(dim):
    return dim.replace('mm', '').strip() if isinstance(dim, str) else dim


In [11]:
def normalize_storage(storage):
    if isinstance(storage, str) and 'TB' in storage:
        return str(int(float(storage.replace('TB', '').strip()) * 1024)) + 'GB'
    return storage

In [12]:
df['Weight (kg)'] = df['Weight'].apply(normalize_weight)
df['Price (INR)'] = df['Price'].apply(normalize_price)
df['Dimensions'] = df['Dimensions'].apply(normalize_dimensions)
df['Storage'] = df['Storage'].apply(normalize_storage)

In [13]:
def rewrite_features(row):
    if pd.isna(row['Battery Life']) or row['Battery Life'].strip().lower() in ['n/a', '']:
        row['Battery Life'] = "Not specified"
    if pd.isna(row['Water Resistance']) or row['Water Resistance'].strip().lower() in ['n/a', '']:
        row['Water Resistance'] = "None"
    if pd.isna(row['Camera']) or row['Camera'].strip().lower() in ['n/a', '']:
        row['Camera'] = "Not available"
    if pd.isna(row['Processor']) or row['Processor'].strip().lower() in ['n/a', '']:
        row['Processor'] = "Not available"
    if pd.isna(row['Material']) or row['Material'].strip().lower() in ['n/a', '']:
        row['Material'] = "Not specified"
    return row


In [14]:
df = df.apply(rewrite_features, axis=1)

In [15]:
df.head(2)

,Product,Weight,Dimensions,Battery Life,Color,Display,Processor,Camera,Water Resistance,Material,Price,Connectivity,Storage,Features,ID,Weight (kg),Price (INR)
0,Galaxy S23,168g,146.3 x 70.9 x 7.6,20 hours,Phantom Black,"6.1 inch Dynamic AMOLED, 2340x1080, 120Hz",Snapdragon 8 Gen 2,"50MP main, 10MP telephoto, 12MP ultra-wide, 12...",IP68,Armor Aluminum,$799,5G,128GB,"Wireless charging, IP68 certified, 5G",1,0.168,Rs.66317
1,QLED TV Q90,18 kg,1230 x 710 x 60,Not specified,Matte Black,"65 inch 4K UHD QLED, HDR10+",Samsung Quantum Processor,Not available,None,Metal & Plastic,$1500,Wi-Fi 5,NaN,"Smart TV, Voice Control, HDR10+",2,18.000,Rs.124500


In [16]:

image_map = {
    'Galaxy S23': 'Galaxy S23.jpg',
    'QLED TV Q90': 'QLED TV Q90.jpg',
    'Galaxy Buds Pro': 'Galaxy Buds Pro.jpg',
    'Galaxy S24': 'Galaxy S24.jpg',
    'Galaxy Z Fold5': 'Galaxy Z Fold5.jpg',
    'Galaxy Watch6': 'Galaxy Watch6.jpg',
    'Galaxy Tab S9': 'Galaxy Tab S9.jpg'
}

 
def enhance_image(image_filename):
    input_path = os.path.join(IMAGE_DIR, image_filename)
    output_path = os.path.join(ENHANCED_IMG_DIR, image_filename)
    try:
        img = Image.open(input_path)
        enhancer = ImageEnhance.Contrast(img)
        enhanced = enhancer.enhance(1.3)
        enhanced.save(output_path)
        return output_path
    except Exception as e:
        print(f"Error processing {image_filename}: {e}")
        return None

df['Enhanced Image Path'] = df['Product'].map(image_map).apply(enhance_image)


In [17]:
df.fillna("Not specified", inplace=True)
enriched_data = df.to_dict(orient='records')
with open(JSON_OUTPUT, 'w', encoding='utf-8') as f:
    json.dump(enriched_data, f, indent=4)


In [18]:
class PDF(FPDF):
    def header(self):
        self.rect(10, 10, 190, 277)
        self.set_font('Arial', 'B', 16)
        self.cell(0, 10, 'Samsung Product Brochure', ln=True, align='C')
        self.ln(10)

    def product_page(self, row):
        self.add_page()
        self.set_font('Arial', 'B', 14)
        self.cell(0, 10, row['Product'], ln=True)
        self.set_font('Arial', '', 11)
        self.ln(5)
        self.multi_cell(0, 8, f"""
Price: {row['Price (INR)']}
Weight: {row['Weight (kg)']} kg
Dimensions: {row['Dimensions']}
Storage: {row['Storage']}
Battery Life: {row['Battery Life']}
Water Resistance: {row['Water Resistance']}
Camera: {row['Camera']}
Processor: {row['Processor']}
Material: {row['Material']}
        """)
        
        self.ln(5)
        if row['Enhanced Image Path'] and os.path.exists(row['Enhanced Image Path']):
            current_y = self.get_y()
            if current_y + 60 < 270:
                self.image(row['Enhanced Image Path'], x=10, y=current_y + 5, w=90)
            else:
                self.add_page()
                self.image(row['Enhanced Image Path'], x=10, y=20, w=90)


pdf = PDF()
pdf.set_auto_page_break(auto=False, margin=15)

for _, row in df.iterrows():
    pdf.product_page(row)

pdf.output(PDF_OUTPUT)

print(" All done! Brochure PDF and JSON file generated.")

 All done! Brochure PDF and JSON file generated.


In [19]:
df

,Product,Weight,Dimensions,Battery Life,Color,Display,Processor,Camera,Water Resistance,Material,Price,Connectivity,Storage,Features,ID,Weight (kg),Price (INR),Enhanced Image Path
0,Galaxy S23,168g,146.3 x 70.9 x 7.6,20 hours,Phantom Black,"6.1 inch Dynamic AMOLED, 2340x1080, 120Hz",Snapdragon 8 Gen 2,"50MP main, 10MP telephoto, 12MP ultra-wide, 12...",IP68,Armor Aluminum,$799,5G,128GB,"Wireless charging, IP68 certified, 5G",1,0.168,Rs.66317,output\enhanced_images\Galaxy S23.jpg
1,QLED TV Q90,18 kg,1230 x 710 x 60,Not specified,Matte Black,"65 inch 4K UHD QLED, HDR10+",Samsung Quantum Processor,Not available,None,Metal & Plastic,$1500,Wi-Fi 5,Not specified,"Smart TV, Voice Control, HDR10+",2,18.000,Rs.124500,output\enhanced_images\QLED TV Q90.jpg
2,Galaxy Buds Pro,6.3g,20 x 18 x 22,8 hours,Silver,Not specified,Not available,Not available,IPX7,Plastic,$199,Bluetooth 5.0,Not specified,"Active Noise Cancellation, Wireless Charging",3,0.006,Rs.16517,output\enhanced_images\Galaxy Buds Pro.jpg
3,Galaxy S24,167g,147.0 x 70.6 x 7.6,22 hours,Titanium Gray,"6.2 inch Dynamic AMOLED 2X, 2340x1080, 120Hz",Snapdragon 8 Gen 3,"50MP main, 10MP telephoto, 12MP ultra-wide, 12...",IP68,Armor Aluminum,$899,5G,256GB,"Gorilla Glass Victus 2, Wireless Charging, IP68",4,0.167,Rs.74617,output\enhanced_images\Galaxy S24.jpg
4,Galaxy Z Fold5,254g,"154.9 x 67.1 x 13.4 (folded), 154.9 x 129.9 x...",4400mAh,Phantom Black,"7.6 inch Dynamic AMOLED 2X (Main), 6.2 inch Dy...",Snapdragon 8 Gen 2 for Galaxy,"50MP main, 12MP ultra-wide, 10MP telephoto, 4M...",IPX8,Armor Aluminum,$1799,5G,256GB,"Foldable Screen, Wireless Charging, IPX8",5,0.254,Rs.149317,output\enhanced_images\Galaxy Z Fold5.jpg
5,Galaxy Watch6,44.4g,44.4 x 43.3 x 9.8,Up to 40 hours,Graphite,"1.4 inch Super AMOLED, 450x450 pixels",Exynos W930,Not available,5ATM + IP68,Armor Aluminum,$349,Bluetooth 5.2,Not specified,"Heart Rate Sensor, Fitness Tracking, GPS",6,0.044,Rs.28967,output\enhanced_images\Galaxy Watch6.jpg
6,Galaxy Tab S9,500g,254.3 x 165.8 x 5.5,10090mAh,Graphite,"11 inch Dynamic AMOLED 2X, 2560x1600, 120Hz",Snapdragon 8 Gen 2,"13MP rear, 12MP front",IP68,Armor Aluminum,$849,Wi-Fi 6E,256GB,"Expandable Storage, S Pen Support",7,0.500,Rs.70467,output\enhanced_images\Galaxy Tab S9.jpg
